In [61]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications import DenseNet121,InceptionV3
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [62]:
# Path to Images Dataset Images
Images_path_part1 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_1'
Images_path_part2 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_2'

# Path to All Images in Dataset
final_path = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/All_Images'

In [63]:

def copy_image_files(src_folder, dest_folder):
    """
    Copy image files from the source folder to the destination folder.

    Parameters:
    src_folder (str): Path to the source folder containing image files.
    dest_folder (str): Path to the destination folder where image files will be copied.

    Returns:
    None
    """
    for filename in os.listdir(src_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            shutil.copy(os.path.join(src_folder, filename), dest_folder)

        
# Create the destination folder if it doesn't exist
if not os.path.exists(final_path):
    os.makedirs(final_path)

copy_image_files(Images_path_part1, final_path)
copy_image_files(Images_path_part2, final_path)

In [64]:
meta_data_df = pd.read_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv')
meta_data_df.sample(5)

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path
9529,HAM_0004350,ISIC_0033970,nv,consensus,40,female,unknown,/Users/darylwanji/Desktop/Computer_Vision_Caps...
5527,HAM_0003718,ISIC_0025233,nv,follow_up,50,male,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...
9107,HAM_0000815,ISIC_0030639,nv,histo,30,female,hand,/Users/darylwanji/Desktop/Computer_Vision_Caps...
2354,HAM_0006203,ISIC_0026456,vasc,histo,60,male,upper extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...
579,HAM_0007607,ISIC_0025883,bkl,histo,75,male,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...


In [65]:
# Add Images' Path to Metadata
meta_data_df['Image_path'] = meta_data_df['image_id'].apply(lambda x: os.path.join(final_path, f"{x}.jpg"))
#Save Meta Data to new file
meta_data_df.to_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv',index=False)

In [66]:
meta_data_df

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
1,HAM_0000118,ISIC_0025030,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
2,HAM_0002730,ISIC_0026769,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
3,HAM_0002730,ISIC_0025661,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
4,HAM_0001466,ISIC_0031633,bkl,histo,75,male,ear,/Users/darylwanji/Desktop/Computer_Vision_Caps...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...


In [67]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder and transform 'Region_Of_Disease' column
meta_data_df['Updated_Diagnosis_Label'] = label_encoder.fit_transform(meta_data_df['Diagnosis'])

# Display the mapping between original labels and encoded labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Encoding Mapping:")
print(label_mapping)

Label Encoding Mapping:
{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}


In [68]:
meta_data_df.sample(5)

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path,Updated_Diagnosis_Label
6350,HAM_0003825,ISIC_0030656,nv,follow_up,30,female,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
4547,HAM_0003074,ISIC_0029816,nv,follow_up,35,male,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
8213,HAM_0003127,ISIC_0028957,nv,histo,35,male,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
6629,HAM_0001144,ISIC_0030673,nv,follow_up,50,male,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
9043,HAM_0001934,ISIC_0026008,nv,histo,65,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5


# Base Model 
This Model will be done without working on the Class Imbalance

In [69]:
# Shuffle Dataset 
# Shuffle the dataset
meta_data_df = meta_data_df.sample(frac=1, random_state=42).reset_index(drop=True)
meta_data_df

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path,Updated_Diagnosis_Label
0,HAM_0007180,ISIC_0033272,mel,histo,65,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...,4
1,HAM_0007195,ISIC_0031923,nv,histo,40,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
2,HAM_0001835,ISIC_0026652,mel,histo,65,male,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,4
3,HAM_0000465,ISIC_0030583,bkl,consensus,35,female,trunk,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2
4,HAM_0001720,ISIC_0034010,nv,histo,45,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
...,...,...,...,...,...,...,...,...,...
10010,HAM_0005460,ISIC_0028199,nv,follow_up,50,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10011,HAM_0005615,ISIC_0031067,nv,follow_up,45,female,trunk,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10012,HAM_0005773,ISIC_0031096,nv,follow_up,45,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10013,HAM_0001053,ISIC_0030998,bkl,consensus,55,female,chest,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2


In [70]:
# Separate features (X) and target variable (y)
X = meta_data_df.drop(columns=['Updated_Diagnosis_Label']) 
y = meta_data_df['Updated_Diagnosis_Label']

# Display the shape of X and y
print("Shape of features (X):", X.shape)
print("Shape of target variable (y):", y.shape)

Shape of features (X): (10015, 8)
Shape of target variable (y): (10015,)


In [71]:
#train-test split without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [72]:
# Data augmentation and generators
train_datagen = ImageDataGenerator(
rescale=1./255,
    brightness_range=[0.8, 1.2],
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=X_train,
    x_col='Image_path',
    y_col='Diagnosis',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_dataframe(
    dataframe=X_val,
    x_col='Image_path',
    y_col='Diagnosis',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=X_test,
    x_col='Image_path',
    y_col='Diagnosis',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 5608 validated image filenames belonging to 7 classes.
Found 1402 validated image filenames belonging to 7 classes.
Found 3005 validated image filenames belonging to 7 classes.


In [73]:
# Load DenseNet121 pre-trained on ImageNet without the top layer , pooling='avg
base_model =DenseNet121(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(7, activation='softmax')(x)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [78]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

from keras.optimizers import Adam
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
            metrics=['accuracy'])

In [79]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# Define callbacks
checkpoint = ModelCheckpoint(filepath='model_weights.keras', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [80]:
# Training the model
history=model.fit(
    train_generator,
    epochs=19,
    batch_size=32,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/19


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

# Predictions
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Metrics
true_classes = test_generator.classes
accuracy = accuracy_score(true_classes, predicted_classes)
cm = confusion_matrix(true_classes, predicted_classes)
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
specificity = cm[0, 0] / (cm[0, 1] + cm[0, 0])
report = classification_report(true_classes, predicted_classes, target_names=list(train_generator.class_indices.keys()), output_dict=True)
precision = report['macro avg']['precision']
f1_score = report['macro avg']['f1-score']
auc = roc_auc_score(tf.keras.utils.to_categorical(true_classes), predictions, multi_class='ovr')

print(f"Accuracy: {accuracy:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-Score: {f1_score:.2f}")
print(f"AUC: {auc:.2f}")